# Intro to Context & Variables

In previous tutorials, all of our skills used the single built-in variable named input though you've often seen it like this: {{$input}}. This built-in variable is used when no specific variable is specified.

For a lot of scenarios, this single input variable is just fine, but Semantic Kernel supports an SKContext object which acts a key-value pair to hold many variables when necessary. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();
KernelBuilder builder = new();
if (settings.Type == "azure")
    builder.AddAzureOpenAIChatCompletion(
        settings.AzureOpenAI.ChatDeployment, "gpt-35-turbo", settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.AddOpenAIChatCompletion(
        settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
Kernel kernel = builder.Build();

The SKContext is a Dictionary<string, TrustAwareString>. For this tutorial, we'll just treat a TrustAwareString like any other string of characters. The first string in the dictionary represents the name of a variable and the second is the value of the variable to be used by the function. You can add many of these key-value pairs and provide the entire SKContext when invoking the function.

In [ ]:
// define your inline function using two variables - and notice we are opting out of using the built-in 'input' variable
string functionDefinition = """
    solve for x and y and show your steps: 
    {{$equation1}}
    {{$equation2}}
    """;

KernelFunction solveFunction = 
    kernel.CreateFunctionFromPrompt(functionDefinition);

// Initialize your prompt parameters
KernelArguments arguments = new()
{
    ["equation1"] = "x + y = 7",
    ["equation2"] = "x - y = 3"
};

FunctionResult result = await kernel.InvokeAsync(
    solveFunction, 
    arguments);

Console.WriteLine(result.GetValue<string>()!);

After running the code above, you can see how the two variables (equation1 and equation2) are replaced by the values stored in the SKContext object. First, the two variable names are inserted into the inline function. Later an SKContext object is created through the CreateNewContext method on the kernel object. Subsequently, an entry is made in the dictionary for each key-value pair. (If you are unfamiliar with dictionaries, think of them like a list of items where each item is defined by a unique 'key' and it's corresponding 'value'. In the code above one item in the dictionary has a key equal to "equation1" and it's corresponding value is "x + y = 7".)

Wait! So how many variables can I have?! Can I define an entire function with just variables?! Let's run another example just for fun.

In [ ]:
// define your inline function using three variables
// and notice we are now using the default 'input' variable
string functionDefinition = """
    {{$fact1}}
    {{$fact2}}
    {{$fact3}}

    {{$input}}
    """;

// create a function from your prompt template
KernelFunction solveFunction =
    kernel.CreateFunctionFromPrompt(functionDefinition);

// initialize your prompt parameters
// define the default 'input' variable in the constructor
KernelArguments arguments = new("In what order did each person finish the race?")
{
    ["fact1"] = "Five people ran a race.",
    ["fact2"] = "Al finished before Bob, but behind Cal.", 
    ["fact3"] = "Dia finished before Elly, but behind Bob."
};

// invoke your function
FunctionResult result = await kernel.InvokeAsync(
    solveFunction,
    arguments);

Console.WriteLine(result.GetValue<string>()!);

As you can see above, you aren't limited by one, two - or even four variables in your function. (You are limited by tokens, but more on that and prompt engineering in another tutorial.)

You can see we first built an SKContext variable and adding several facts to that dictionary. The variable names are replaced by the actual strings so it doesn't really matter what the dictionary key is so long as it makes sense to you the developer! Both the dictionary's key and value must both be strings - no integers, floats, etc. You can store all of these as string of course. You can even serialize objects into variables and then deserialize them in native functions.

# Exercise

Define and run a semantic function below that uses multiple variables. Your function should support at least three different names of objects. Your semantic function should ask OpenAI to return what is most common across all of your objects.